In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import fsolve
from scipy.optimize import root
from bokeh.charts import Bar,Histogram, output_notebook, show , BoxPlot
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import weibull_min
from scipy.stats import chisquare,chi2
from scipy.stats import probplot
from bokeh.layouts import row
from scipy import stats
import pylab 
TOOLS = 'box_zoom,box_select,resize,reset,save'

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
df_turbine = pd.read_csv('/Users/sroy/Personal/Mathematical_statistics/Data/Turbine.csv')
# Reading few lines
wind = df_turbine['AveSpeed']

In [4]:
def weibull_shape(k,data):
    numer = sum((data**k)*np.log(data))
    denom = sum((data**k))
    return numer/denom - 1/k - np.log(data).mean()
def weibull_scale(k,data):
    return ((data**k).mean())**(1/k)

In [5]:
# Estomating shape parameter
k = root(weibull_shape, x0=1, args =(wind))['x']
print(k)
# Estimating scale parameter
print(weibull_scale(k,wind))
lam = weibull_scale(k,wind)

[ 3.16932037]
[ 7.6612776]


In [6]:
# output_notebook()
# max_x = wind.max()+2
# p = Histogram(wind , title= 'Distribution of average wind speeds', density = True,
#               xlabel= 'meters/sec', ylabel = 'Density',bins= list(range(2,int(max_x),1)))
# show(p)


In [16]:
max_x = wind.max()+1
p = figure(title="Distribution of average wind speeds", tools="save", background_fill_color="#E8DDCB")
hist,edges = np.histogram(wind, density=True, bins= list(range(2,int(max_x),1)))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
x = np.linspace(2, 16, 1000)
pdf = (k/lam)*(x/lam)**(k-1) * np.exp(-(x/lam)**k)
p.line(x, pdf, line_color="Orange", line_width=8, alpha=0.7, legend="PDF")
p.xaxis.axis_label = 'meters/sec'
p.yaxis.axis_label = 'Density'
show(p)

In [17]:
ecdf=ECDF(wind)
cdf = 1 - np.exp(-(x/lam)**k)
p = figure(title="ECDF of wind data", tools=TOOLS, background_fill_color="#E8DDCB")
p.line(ecdf.x,ecdf.y, legend = "ECDF")
p.line(x, cdf, line_color="black", line_width=2, alpha=0.7, legend="CDF")
p.xaxis.axis_label = 'x'
p.yaxis.axis_label = 'Fn(x)'
show(p)

In [9]:
# Now for the chi-square goodness of fit test
# Get the deciles
q = weibull_min.ppf(np.arange(0.1,1,0.1),c=3.169, scale=7.661)
q = np.concatenate(([0],q,[14]),axis=0)
# Get the counts in each sub-interval
print(np.histogram(wind,bins=q)[0])
count = np.histogram(wind,bins=q)[0]
expected = len(wind)* 0.1
# compute chi-square test statistic
print(sum((count - expected)**2/expected))

# computing p value

expected_array = np.ones(len(count))*16.8
# directly 
print(chisquare(count,expected_array, ddof=2)[1])
# using cdf
p_value = 1 - stats.chi2.cdf(x=3.071,df=7)
print(p_value)


[17 18 19 13 20 14 17 17 14 19]
3.07142857143
0.878316781662
0.878357352279


In [18]:
#-----------------------------------------
# Example 6.13
# Simulation comparing two estimators for uniform

my_mean = np.zeros(1000)
my_max  = np.zeros(1000)
s = np.random.uniform(0,1,25)
for i in range(1000):
    x = np.random.uniform(0,12,25)
    my_mean[i] = 2*np.mean(x)
    my_max[i] = (26/25)*np.max(x)
print(my_mean.mean())
print(my_mean.std())

print(my_max.mean())
print(my_max.std())

p1 = Histogram(my_mean, density = False,xlabel= 'means', ylabel = 'Frequency')
p2 = Histogram(my_max,density = False,xlabel= 'max', ylabel = 'Frequency')
show(row(p1,p2))

12.0322008376
1.41827136585
11.9926303352
0.490139571715


In [29]:
n = len(wind)
theta1 = (wind > 5).mean()
theta2 = 1 - weibull_min.cdf(5,c=3.169, scale=7.661)
eta1 = wind.quantile(0.1)
eta2 = weibull_min.ppf(0.1,c=3.169, scale=7.661)
print(theta1,theta2,eta1,eta2)
B = 10**4
# nonparametric bootstrap to find standard errors
boot_theta1 = np.zeros(B)
boot_shape = np.zeros(B)
boot_scale = np.zeros(B)
boot_theta2 = np.zeros(B)
boot_eta1 = np.zeros(B)
boot_eta2 = np.zeros(B)
for i in range(B):
    boot_wind = wind.sample(n, replace=True)
    boot_theta1[i] = (boot_wind > 5).mean()
    boot_shape[i] = root(weibull_shape, x0=1, args =(boot_wind))['x']
    boot_scale[i] = weibull_scale( boot_shape[i],boot_wind)
    boot_theta2[i] = 1 - weibull_min.cdf(5,c= boot_shape[i], scale= boot_scale[i])
    boot_eta1[i]  = boot_wind.quantile(0.1)
    boot_eta2[i] = weibull_min.ppf(0.1,c=boot_shape[i], scale= boot_scale[i])

0.75 0.772082709857 3.77 3.76603775449


In [30]:
series1 = stats.probplot(boot_theta1, dist="norm")
series2 = stats.probplot(boot_theta2, dist="norm")
p1 = figure(title="Normal QQ-Plot", background_fill_color="#E8DDCB")
p1.scatter(series1[0][0],series1[0][1], fill_color="red")
p2 = figure(title="Normal QQ-Plot", background_fill_color="#E8DDCB")
p2.scatter(series2[0][0],series2[0][1], fill_color="red")

series3 = stats.probplot(boot_eta1, dist="norm")
series4 = stats.probplot(boot_eta2, dist="norm")
p3 = figure(title="Normal QQ-Plot", background_fill_color="#E8DDCB")
p3.scatter(series3[0][0],series3[0][1], fill_color="red")
p4 = figure(title="Normal QQ-Plot", background_fill_color="#E8DDCB")
p4.scatter(series4[0][0],series4[0][1], fill_color="red")



p1.xaxis.axis_label = 'Theoritical Quantiles'
p1.yaxis.axis_label = 'Sample Quantiles'

p2.xaxis.axis_label = 'Theoritical Quantiles'
p2.yaxis.axis_label = 'Sample Quantiles'


p3.xaxis.axis_label = 'Theoritical Quantiles'
p3.yaxis.axis_label = 'Sample Quantiles'


p4.xaxis.axis_label = 'Theoritical Quantiles'
p4.yaxis.axis_label = 'Sample Quantiles'

show(gridplot(p1,p2,p3,p4, ncols=2, plot_width=400, plot_height=400, toolbar_location=None))

In [26]:
print(boot_theta1.std(),boot_theta2.std(), boot_theta1.var()/boot_theta2.var())

print(boot_eta1.std(),boot_eta2.std(), boot_eta1.var()/boot_eta2.var())

0.033279538735 0.0237108029387 1.96998116638
1.7763568394e-15 0.186022940374 9.11858425656e-29


In [ ]:
sd(boot.eta1) # 0.2161054
sd(boot.eta2) # 0.1839509
var(boot.eta1) / var(boot.eta2) # 1.380154